<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

## Lab 4.2.2: Feature Selection

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### 5. Forward Feature Selection

> Forward Selection: Forward selection is an iterative method in which we start with having no feature in the model. In each iteration, we keep adding the feature which best improves our model till an addition of a new variable does not improve the performance of the model.

Create a Regression model using Forward Feature Selection by looping over all the features adding one at a time until there are no improvements on the prediction metric ( R2  and  AdjustedR2  in this case).

#### 5.1 Load Wine Data & Define Predictor and Target

In [17]:
from sklearn.linear_model import LinearRegression as lr
from sklearn.model_selection import train_test_split
print(lr)

<class 'sklearn.linear_model._base.LinearRegression'>


In [18]:
## Load the wine quality dataset

# Load the wine dataset from csv
wine = pd.read_csv('/Users/faez/Downloads/git-institute-of-data/module4/data/winequality_merged.csv')

# define the target variable (dependent variable) as y
y = wine['quality']

# Take all columns except target as predictor columns
predictor_columns = [c for c in wine.columns if c != 'quality']
# Load the dataset as a pandas data frame
X = pd.DataFrame(wine, columns = predictor_columns)

In [25]:
## Create training and testing subsets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(y_test) 
print(X_test)

4973    5
2499    7
1017    6
2763    6
4428    5
       ..
486     5
6035    6
2556    7
6077    5
2553    6
Name: quality, Length: 1300, dtype: int64
      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
4973            5.9              0.19         0.21             1.7      0.045   
2499            6.6              0.20         0.38             7.9      0.052   
1017            8.0              0.18         0.37             0.9      0.049   
2763            8.2              0.23         0.49             0.9      0.057   
4428            7.1              0.26         0.32            16.2      0.044   
...             ...               ...          ...             ...        ...   
486            10.2              0.67         0.39             1.9      0.054   
6035            6.7              0.28         0.28             4.5      0.051   
2556            6.6              0.23         0.32             0.9      0.041   
6077            6.4              0.32 

#### 5.2 Overview of the code below

The external `while` loop goes forever until there are no improvements to the model, which is controlled by the flag `changed` (until is **not** changed).
The inner `for` loop goes over each of the features not yet included in the model and calculates the correlation coefficient. If any model improves on the previous best model then the records are updated.

#### Code variables
- `included`: list of the features (predictors) that were included in the model; starts empty.
- `excluded`: list of features that have **not** been included in the model; starts as the full list of features.
- `best`: dictionary to keep record of the best model found at any stage; starts 'empty'.
- `model`: object of class LinearRegression, with default values for all parameters.

#### Methods of the `LinearRegression` object to investigate
- `fit()`
- `fit.score()`

#### Adjusted $R^2$ formula
$$Adjusted \; R^2 = 1 - { (1 - R^2) (n - 1)  \over n - k - 1 }$$

#### Linear Regression [reference](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)

In [22]:
## Flag intermediate output

show_steps = True   # for testing/debugging
# show_steps = False  # without showing steps

In [28]:
## Use Forward Feature Selection to pick a good model

# start with no predictors
included = []
# keep track of model and parameters
best = {'feature': '', 'r2': 0, 'a_r2': 0}
# create a model object to hold the modelling parameters
model = lr() # create a model for Linear Regression
# get the number of cases in the training data
n = X_train.shape[0]

while True:
    changed = False
    
    if show_steps:
        print('') 

    # list the features to be evaluated
    excluded = list(set(X.columns) - set(included))
    
    if show_steps:
        print('(Step) Excluded = %s' % ', '.join(excluded))  

    # for each remaining feature to be evaluated
    for new_column in excluded:
        
        if show_steps:
            print('(Step) Trying %s...' % new_column)
            print('(Step) - Features = %s' % ', '.join(included + [new_column]))

        # fit the model with the Training data
        # the fit method modifies the regression model and returns a reference to itself
        fit = model.fit(X_train[included + [new_column]], y_train)
        # calculate the score (R^2 for Regression)
        r2 = fit.score(X_train[included + [new_column]], y_train)
        # number of predictors in this model
        k = len(included) + 1
        # calculate the adjusted R^2
        adjusted_r2 = 1 - ( ( (1 - r2) * (n - 1) ) / (n - k - 1) )

        if show_steps:
            print('(Step) - Adjusted R^2: This = %.3f; Best = %.3f' % 
                  (adjusted_r2, best['a_r2']))

        # if model improves
        if adjusted_r2 > best['a_r2']:
            # record new parameters
            best = {'feature': new_column, 'r2': r2, 'a_r2': adjusted_r2}
            # flag that found a better model
            changed = True
            if show_steps:
                print('(Step) - New Best!   : Feature = %s; R^2 = %.3f; Adjusted R^2 = %.3f' % 
                      (best['feature'], best['r2'], best['a_r2']))
    # END for

    # if found a better model after testing all remaining features
    if changed:
        # update control details
        included.append(best['feature'])
        excluded = list(set(excluded) - set(best['feature']))
        print('Added feature %-4s with R^2 = %.3f and adjusted R^2 = %.3f' % 
              (best['feature'], best['r2'], best['a_r2']))
    else:
        # terminate if no better model
        break

print('')
print('Resulting features:')
print(', '.join(included))


(Step) Excluded = density, red_wine, sulphates, volatile acidity, pH, alcohol, residual sugar, total sulfur dioxide, citric acid, fixed acidity, free sulfur dioxide, chlorides
(Step) Trying density...
(Step) - Features = density
(Step) - Adjusted R^2: This = 0.098; Best = 0.000
(Step) - New Best!   : Feature = density; R^2 = 0.098; Adjusted R^2 = 0.098
(Step) Trying red_wine...
(Step) - Features = red_wine
(Step) - Adjusted R^2: This = 0.014; Best = 0.098
(Step) Trying sulphates...
(Step) - Features = sulphates
(Step) - Adjusted R^2: This = 0.002; Best = 0.098
(Step) Trying volatile acidity...
(Step) - Features = volatile acidity
(Step) - Adjusted R^2: This = 0.072; Best = 0.098
(Step) Trying pH...
(Step) - Features = pH
(Step) - Adjusted R^2: This = 0.000; Best = 0.098
(Step) Trying alcohol...
(Step) - Features = alcohol
(Step) - Adjusted R^2: This = 0.205; Best = 0.098
(Step) - New Best!   : Feature = alcohol; R^2 = 0.205; Adjusted R^2 = 0.205
(Step) Trying residual sugar...
(Step) 

(Step) - Adjusted R^2: This = 0.296; Best = 0.299
(Step) Trying fixed acidity...
(Step) - Features = alcohol, volatile acidity, sulphates, residual sugar, red_wine, density, free sulfur dioxide, fixed acidity
(Step) - Adjusted R^2: This = 0.296; Best = 0.299
(Step) Trying chlorides...
(Step) - Features = alcohol, volatile acidity, sulphates, residual sugar, red_wine, density, free sulfur dioxide, chlorides
(Step) - Adjusted R^2: This = 0.296; Best = 0.299
Added feature total sulfur dioxide with R^2 = 0.300 and adjusted R^2 = 0.299

(Step) Excluded = chlorides, citric acid, pH, fixed acidity
(Step) Trying chlorides...
(Step) - Features = alcohol, volatile acidity, sulphates, residual sugar, red_wine, density, free sulfur dioxide, total sulfur dioxide, chlorides
(Step) - Adjusted R^2: This = 0.300; Best = 0.299
(Step) - New Best!   : Feature = chlorides; R^2 = 0.301; Adjusted R^2 = 0.300
(Step) Trying citric acid...
(Step) - Features = alcohol, volatile acidity, sulphates, residual sugar



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



